# Notebook for tidying data

Intended for loading and working with the raw dataframe to avoid having to scrape all the data each time. 

## Importing and loading stuff

In [1]:
import requests,os,time,datetime,json,ast
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
data_local_path = r"C:\Users\Troel\Lecture slides etc\Exam\Ny mappe\SDS19-Exam-Project\data_16-07_19_04.csv"
out_csv = "DataFrame_16-19_with_dicts.csv"

In [3]:
org_raw_data = pd.read_csv(data_local_path, sep=',', low_memory = False)
raw_data = org_raw_data.copy()

## Working with the data

In [4]:
# Defining empty dict entries
nan_loc_val = {'latitude': None, 'street': {'id': None, 'name': None}, 'longitude': None}
nan_outcome_val = {'id': None, 'name': None}

In [5]:
#Transforming strings to dicts
raw_data['location_as_dict'] = [ast.literal_eval(i) if pd.notnull(i) == True else nan_loc_val for i in tqdm(raw_data['location'])]
raw_data['outcome_object_as_dict'] = [ast.literal_eval(i) if pd.notnull(i) == True else nan_val for i in tqdm(raw_data['outcome_object'])]

In [6]:
loc_keys = list(raw_data['location_as_dict'][1].keys())
obj_keys = list(raw_data['outcome_object_as_dict'][1].keys())
street_keys = list(raw_data['location_as_dict'][1]['street'].keys())

In [7]:
#Splitting into columns
for key in tqdm(loc_keys):
    raw_data['{}'.format(key)] = [raw_data['location_as_dict'][i][key] for i in tqdm(range(len(raw_data['location_as_dict'])))]

for key in tqdm(obj_keys):
    raw_data['outcome_object_{}'.format(key)] = [raw_data['outcome_object_as_dict'][i][key] for i in tqdm(range(len(raw_data['outcome_object_as_dict'])))]


In [8]:
for key in tqdm(street_keys):
    raw_data.insert(loc = raw_data.columns.get_loc('street')+1, column='street_{}'.format(key), value=[raw_data['location_as_dict'][i]['street'][key] for i in tqdm(range(len(raw_data['outcome_object_as_dict'])))])

## Remove unnecessary columns

In [9]:
#Removing str columns as intermediary columns
tidy_data = raw_data.drop(columns= ['location','outcome_object','street','location_as_dict','outcome_object_as_dict'])

## Save new dataframe

In [10]:
tidy_data.to_csv(out_csv,index = False)